# Agglomerative Info-Clustering

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ccha23/Agglomerative-Info-Clustering/master?urlpath=lab/tree/AIC.ipynb)

This notebook demonstrates the agglomerative info-clustering algorithm (AIC) implemented in C++. It can be run using the [xeus-cling](https://xeus-cling.readthedocs.io/en/latest/) C++11 jupyter kernel.

## Info-Clustering

$\def\M#1{\boldsymbol{#1}}
\def\R#1{\mathsf{#1}}
\def\RM#1{\M{\R{#1}}}
\def\mc#1{\mathcal{#1}}
\def\Set#1{\left\{{#1}\right\}}
\def\abs#1{\left|{#1}\right|}
\def\RZ{\R{Z}}
\def\Rx{\R{x}}
\def\RX{\R{X}}
\def\RMx{\RM{x}}
\def\MM{\M{M}}
\def\MS{\M{S}}
\def\mcP{\mathcal{P}}
$*Info-clustering (IC)* is a hierarchical clustering using *multivariate mutual information (MMI)* as the cophenetic similarity.

**Definition** (MMI) For any tuple of random variables
$$\RZ_V:=(\RZ_i\mid i\in V),$$
the multivariate mutual information is defined as
$$\begin{align}I(\RZ_V) &:= \min\{\gamma\geq 0|\exists \mcP\in \Pi'(V),\tag{1a}\\
&\kern6em \left.\sum_{B\in \mcP} [H(\RZ_B)-\gamma] = H(\RZ_C)-\gamma\right\}\tag{1b}\label{eq:rir}\\
&= \min_{\mcP\in \Pi'(V)} \frac{D(P_{\RZ_V}\|\prod_{C\in \mcP} P_{\RZ_C})}{\abs{\mcP}-1} \tag{2}\label{eq:mmi}\end{align}$$ 
where $\Pi'(V)$ is the collection of partitions of $V$ into at least 2 non-empty disjoint sets. \eqref{eq:mmi} extends the formula 
$$I(\RZ_i\wedge \RZ_j)=D(P_{\RZ_i,\RZ_j}\|P_{\RZ_i}P_{\RZ_j})$$ 
of mutual information in terms of KL-divergence. The product distribution corresponds to and independence relation in the multivariate case. \eqref{eq:rir} means that $\gamma$ is the amount of information removal of which leads to the independence relation. $\square$

**Definition** (IC) Given a source $\RZ_V$ where $V$ is the ground set (a finite set of objects to cluster), the set of clusters at similarity $\gamma\in \mathbb{R}$ given by info-clustering is
$$\begin{align}\begin{split}\mc{C}_{\gamma}(\RZ_V):=\{C\subseteq V\mid &I(\RZ_C)>\gamma\\
&I(\RZ_{C'})\leq \gamma\quad \forall C'\subseteq V:C\subsetneq C'\},\end{split}\tag{3}\label{eq:IC}\end{align} $$
namely, the set of all inclusion-wise maximal non-singleton subsets MMI exceeding $\gamma$. The singletons $\Set{i}$ for $i\in V$ may be included as trivial clusters by setting $I(\RZ_i)=\infty$. $\square$

**Proposition** For any $\RZ_V$, the info-clustering solution is unique and polynomial-time computable.

 The above follows from the submodularity of the entropy function:
 $$h(B_1)+h(B_2)\geq h(B_1\cup B_2) + h(B_1\cap B_2)\quad \forall B_1,B_2\subseteq V.\tag{4}\label{eq:submodular}$$
 where 
 $$\begin{align}h:B\subseteq V\mapsto H(\RZ_B)\tag{5}\label{eq:h}\end{align}$$
 is the entropy function of $\RZ_V$ defined using Shannon's entropy or differential entropy $H$. $\square$

## Agglomerative clustering

An *agglomerative info-clustering (AIC)* method merges clusters recursively into larger ones starting from the singletons. An exact solution can be calculated using *submodular function optimization techniques* while an approximate solution can be obtained using the *single-linkage method*. Execute the following code to set up the paths.

In [1]:
#pragma cling add_include_path("include") // main include path

### Exact solution

Execute the following cell to load the library for AIC.

In [2]:
#include <IC/AIC>     // agglomerative info-clustering

To specify the source $\RZ_V$ for clustering, it suffices to specify the entropy function since the divergence and therefore MMI can be computed from the function:
$$
\begin{align}
I(\RZ_V) &= \min_{\mcP\in \Pi'(V)} \frac{h(C) - h(V)}{\abs{\mcP}-1}\tag{6a}\\
I(\RZ_B\wedge \RZ_C) &= h(B) + h(C) - h(B\cup C)\qquad B,C\subseteq V: B\cap C=\emptyset
\tag{6b}\\
\end{align}
$$
N.b., if $B\cap C\neq \emptyset$, the equality $H(\RZ_B,\RZ_C)=H(\RZ_{B\cup C})$ may not hold for continuous random variables. E.g., if $\RZ$ is a standard gaussian random variable, we have $h(\RZ)=\frac12\log_2(2\pi e)$ but $I(\RZ\wedge \RZ)=\infty$ as $h(\RZ,\RZ)=-\infty$. 

As an example, we define the entropy function for
- $V=\Set{0,1,2}$ is the ground set,
- $\RZ_0=0$ is a constant, and 
- $\RZ_1=\RZ_2$ is a uniformly random bit.

In [3]:
class H1 : public IC::SF {
    public:
    double operator() (const std::vector<size_t> &B) const {
        int entropy = 0;
        if (std::find(B.begin(),B.end(),1)!=B.end() || 
            std::find(B.begin(),B.end(),2)!=B.end())
            entropy++;
        return entropy;
    }

    size_t size() const { return 3; }
};
    
H1 h1;

The entropy function can be evaluated over all subsets of the ground set. The pairwise mutual information can be computed. 

In [4]:
[](IC::SF &h) {
    std::vector<std::vector<size_t>> power_set = {{}, // emptyset
                                                 {0},{1},{2}, // singletons
                                                 {0,1},{0,2},{1,2},
                                                 {0,1,2}};
    for (size_t i=0; i<power_set.size(); i++) {
        std::cout << "h(" << power_set[i] << ") = " 
                  << h(power_set[i]) << std::endl;
    }
    
    for (size_t i=0; i<h.size(); i++)
        for (size_t j=i+1; j<h.size(); j++)
        std::cout << "I(Z" << i << "; Z" << j << ") = " 
                  << h.mutual_info(i,j) << std::endl;
} (h1);

h([ ]) = 0
h([ 0 ]) = 0
h([ 1 ]) = 1
h([ 2 ]) = 1
h([ 0 1 ]) = 1
h([ 0 2 ]) = 1
h([ 1 2 ]) = 1
h([ 0 1 2 ]) = 1
I(Z0; Z1) = 0
I(Z0; Z2) = 0
I(Z1; Z2) = 1


In general, it takes exponential time to go through all values of the entropy function one by one. Fortunately, the exact clustering solution can be obtained without going through all the values, owing to the submodularity of entropy. The solution can also be computed gradually in an agglomerative fashion.

In [5]:
IC::AIC psp1(h1); // The data structure for AIC (inherits IC::HC)

[](IC::AIC &psp) {
    std::cout << psp.getCoarsestPartition() 
              << std::endl; // get coarsest partition generated
    while (psp.agglomerate()) // agglomerate until partition {V}
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
} (psp1);

[ [ 0 ] [ 1 ] [ 2 ] ]
agglomerates to [ [ 0 ] [ 1 2 ] ] at critical value 1
agglomerates to [ [ 0 1 2 ] ] at critical value -0


The non-singleton subsets are the desired clusters of the info-clustering solution.

In [6]:
[](IC::AIC &psp) {
    std::vector<double> psp_gamma = psp.getCriticalValues();
    std::cout << "Critical values of MMI: " << psp_gamma << std::endl;
    
    psp_gamma.insert(psp_gamma.begin(), INFINITY);
    psp_gamma.push_back(-INFINITY);

    for (size_t i=1; i < psp_gamma.size(); i++) {
        std::cout << "Clusters with MMI in (" << psp_gamma[i] 
                  << ","               << psp_gamma[i-1]<< "] : ";
        for (std::vector<size_t> C : psp.getPartition(psp_gamma[i]))
            if (C.size()>1)
                std::cout << C << " ";
        std::cout << std::endl;
    }
} (psp1);

Critical values of MMI: [ 1 -0 ]
Clusters with MMI in (1,inf] : 
Clusters with MMI in (-0,1] : [ 1 2 ] 
Clusters with MMI in (-inf,-0] : [ 0 1 2 ] 


We can return only clusters containing a certain node as follows.

In [7]:
[](IC::AIC &psp, size_t v) {
    std::vector<double> psp_gamma = psp.getCriticalValues(v);
    std::cout << "Critical values of MMI for Node "<< v
              << " : " << psp_gamma << std::endl;
    
    psp_gamma.insert(psp_gamma.begin(), INFINITY);
    psp_gamma.push_back(-INFINITY);

    for (size_t i=1; i < psp_gamma.size(); i++) {
        std::cout << "Cluster containing Node " << v 
                  << " with MMI in (" << psp_gamma[i] 
                  << "," << psp_gamma[i-1] << "] : " 
                  << psp.getCluster(v,psp_gamma[i]) 
                  << std::endl;
    }
} (psp1,1);

Critical values of MMI for Node 1 : [ 1 -0 ]
Cluster containing Node 1 with MMI in (1,inf] : [ 1 ]
Cluster containing Node 1 with MMI in (-0,1] : [ 1 2 ]
Cluster containing Node 1 with MMI in (-inf,-0] : [ 1 2 0 ]


The similarity between a pair of nodes is the maximum MMI for a cluster containing both nodes, i.e.,
$$\operatorname{sim}(i,j):=\sup_{C\subseteq V:i,j\in C} I(\RZ_C)\tag{7}$$

In [8]:
[](IC::SF &h, IC::AIC &psp) {
    for (size_t i=0; i < h.size(); i++)
        for (size_t j=0; j < h.size(); j++)
            std::cout << "sim(" << i << "," << j << ")="
                      << psp.similarity(i,j) << std::endl;
} (h1,psp1);

sim(0,0)=inf
sim(0,1)=-0
sim(0,2)=-0
sim(1,0)=-0
sim(1,1)=inf
sim(1,2)=1
sim(2,0)=-0
sim(2,1)=1
sim(2,2)=inf


### Approximate solution

If the MMI is computed with *Chow-Liu (CL)* tree approximation, the method reduces to the single-linkage method on the mutual mutual information relevance network (MIRN), which is a weighted graph on $V$ with edge weight $I(\RZ_i\wedge \RZ_j)$. It can also be viewed as an agglomerative method with the maximum mutual information
$$
\max_{i\in C,j\in C'} I(\RZ_i\wedge \RZ_j)
$$
as a measure of similarity between two clusters $C$ and $C'$.

Execute the following cell to load the CL approximation.

In [9]:
#include <IC/ChowLiu> // Chow-Liu tree approximation

The following gives the Chow-Liu tree approximation.

In [10]:
[](IC::SF &h) {
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }    
} (h1);

critical values : [ 1 0 ]
partition at threshold 1:[ [ 0 ] [ 1 ] [ 2 ] ]
partition at threshold 0:[ [ 0 ] [ 1 2 ] ]


The MIRN can be used as the input.

In [11]:
[] {// Define the MIRN
    std::vector<size_t> first_node = {0,1},  // list of 1st nodes i in edges
                        second_node = {1,2}; // list of 2nd nodes j in edges
    std::vector<double> gamma = {0,1};       // list of edge weights I(Zi;Zj)
    // Single-linkage
    IC::CL cl(3, first_node, second_node, gamma);

    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }    
} ();

critical values : [ 1 0 ]
partition at threshold 1:[ [ 0 ] [ 1 ] [ 2 ] ]
partition at threshold 0:[ [ 0 ] [ 1 2 ] ]


## Examples on different source models

For convenience, the entropy functions for several common source models have been implemented.

### Hypergraphical source

A hypergraphical source is defined as follows:
$$
\begin{align}
\RZ_i &= (\RX_e|e\in E, i\in \phi(e))\qquad i\in V\\
\end{align}
$$
for some given weighted hypergraph on $V$ with edge set $E$ where, for each edge $e\in E$,
- $\phi(e)$ is the set of incident vertices on $e\in E$, and
- $H(\RX_e)$ is the edge weight where $\RX_e$ for $e\in E$ are independent edge random variables.

The entropy function simplifies to
$$
h(B):=\sum_{e\in E: \phi(e)\cap B\neq \emptyset} H(\RX_e)\qquad B\subseteq V.
$$

To load the library for hypergraphical source:

In [12]:
#include <IC/hypergraph>

The earlier example is a *pairwise independent network model (PIN)*, which is a special case of the hypergraphical source model when the hypergraph is a graph. More precisely, with $V=\Set{0,1,2}$,
$$
\begin{align}
\RZ_0 &= 0\\
\RZ_1 &= \RX_a\\
\RZ_2 &= \RX_a
\end{align}
$$
and the edge set $E := \Set{a}$ contains a single edge $a$ with incident vertices in $\phi(a)=\Set{1,2}$ and edge weight $H(\RX_a):=1$.

In [13]:
{
    // define the incidence matrix
    Eigen::MatrixXd M(3,1); // 3 vertices and 1 edge
    M << 0, 
         1, 
         1;
    std::cout << "Incidence matrix of the (weighted) hypergraph: " 
              << std::endl << "M= \n" << M << std::endl;
    IC::HypergraphEntropy h(M);
    
    // compute pairwise mutual information
    std::cout << std::endl << "Pairwise mutual information: " << std::endl;
    for (size_t i=0; i<h.size(); i++)
        for (size_t j=i+1; j<h.size(); j++)
        std::cout << "I(Z" << i << "; Z" << j << ") = " 
                  << h.mutual_info(i,j) << std::endl;    
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }   
}

Incidence matrix of the (weighted) hypergraph: 
M= 
0
1
1

Pairwise mutual information: 
I(Z0; Z1) = 0
I(Z0; Z2) = 0
I(Z1; Z2) = 1

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] ]
agglomerates to [ [ 0 ] [ 1 2 ] ] at critical value 1
agglomerates to [ [ 0 1 2 ] ] at critical value -0

Approximate solution: 
critical values : [ 1 0 ]
partition at threshold 1:[ [ 0 ] [ 1 ] [ 2 ] ]
partition at threshold 0:[ [ 0 ] [ 1 2 ] ]


The following is an example beyond the PIN model because of the hyperedge covering the first three nodes.

In [14]:
{
    // define the incidence matrix
    Eigen::MatrixXd M(6,4); // 6 vertices and 4 hyperedges
    M << 1, 1, 0, 0,
         1, 1, 0, 0,
         1, 0, 0, 0,
         0, 0, 1, 0,
         0, 0, 1, 0,
         0, 0, 0, 1;
    std::cout << "Incidence matrix of the (weighted) hypergraph: " 
              << std::endl << "M= \n" << M << std::endl;
    IC::HypergraphEntropy h(M);
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }   
}

Incidence matrix of the (weighted) hypergraph: 
M= 
1 1 0 0
1 1 0 0
1 0 0 0
0 0 1 0
0 0 1 0
0 0 0 1

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ]
agglomerates to [ [ 0 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ] at critical value 2
agglomerates to [ [ 0 1 2 ] [ 3 4 ] [ 5 ] ] at critical value 1
agglomerates to [ [ 0 1 2 3 5 4 ] ] at critical value -0

Approximate solution: 
critical values : [ 2 1 0 ]
partition at threshold 2:[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ]
partition at threshold 1:[ [ 0 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ]
partition at threshold 0:[ [ 0 1 2 ] [ 3 4 ] [ 5 ] ]


The following hypergraph is *minimally connected* in the sense that removing any hyperedge disconnects the hypergraph.

In [15]:
{
    // define the incidence matrix
    Eigen::MatrixXd M(6,3); // 6 vertices and 3 hyperedges
    M << 1, 0, 1,
         1, 1, 0,
         0, 1, 1,
         1, 0, 0,
         0, 1, 0,
         0, 0, 1;
    std::cout << "Incidence matrix of the (weighted) hypergraph: " 
              << std::endl << "M= \n" << M << std::endl;
    IC::HypergraphEntropy h(M);
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }   
}

Incidence matrix of the (weighted) hypergraph: 
M= 
1 0 1
1 1 0
0 1 1
1 0 0
0 1 0
0 0 1

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ]
agglomerates to [ [ 0 2 1 ] [ 3 ] [ 4 ] [ 5 ] ] at critical value 1.5
agglomerates to [ [ 0 2 1 3 4 5 ] ] at critical value 1

Approximate solution: 
critical values : [ 1 ]
partition at threshold 1:[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] ]


### Gaussian source

For the (jointly) gaussian source model,
$$\RZ_i := \MM_i\RMx \qquad \text{for $i\in V$}$$
where $\RMx$ is a while gaussian random row vector of any given length and $\MM_i$ can be any (deterministic) matrix of any given number of rows.

The entropy function is given by
$$
\begin{align}
\begin{split}
h(B)&:=\frac{k}2\log_2 2\pi e \det \MM_B\MM_B^{\intercal}\qquad \text{where}\\
\MM_B &:=\begin{bmatrix}\MM_{i_1} \\ \MM_{i_2}\\ \vdots\end{bmatrix}\kern1em \text{for any}\kern1em B=\Set{i_1,i_2,\dots}\subseteq V.
\end{split}\label{eq:gaussian}
\end{align}
$$
$\MM$ is obtained by stacking the matrices $\MM_i$ for $i\in B$ vertically.
Note that the above determinant is undefined if $\MM$ loses rank. Such degeneracy is not handled automatically by the library. Instead, one can cluster based on the matrix rank first with 
$$h(B):=\operatorname{rank}(\MM_B)$$ 
and then remove the linear dependency for further clustering using the gaussian model.  

To load the library for the gaussian source model:

In [16]:
#include <IC/gaussian>

The following is a simple example with three users in $V=\Set{0,1,2}$ where:
$$
\begin{aligned}
\RZ_0 &= \Rx_0 + \Rx_1\\
\RZ_1 &= \Rx_0 + \Rx_2\\
\RZ_2 &= \Rx_3,
\end{aligned}
\qquad \text{for the white gaussian vector }
\RMx = \begin{bmatrix} \Rx_1 \\ \Rx_2 \\ \Rx_3 \end{bmatrix}.
$$

In [17]:
{
    // define the linear mappings
    Eigen::MatrixXd M0(1,4), M1(1,4), M2(1,4);
    M0 << 1,1,0,0;
    M1 << 1,0,1,0;
    M2 << 0,0,0,1;
    std::cout << "Linear mappings: " 
          << std::endl << "M0= \n" << M0 << std::endl
          << std::endl << "M1= \n" << M1 << std::endl
          << std::endl << "M2= \n" << M2 << std::endl;
    std::vector<Eigen::MatrixXd> LinearMappings = {M0,M1,M2};
    IC::GaussianEntropy h(LinearMappings);
    
    // compute pairwise mutual information
    std::cout << std::endl << "Pairwise mutual information: " << std::endl;
    for (size_t i=0; i<h.size(); i++)
        for (size_t j=i+1; j<h.size(); j++)
        std::cout << "I(Z" << i << "; Z" << j << ") = " 
                  << h.mutual_info(i,j) << std::endl;    
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }   
}

Linear mappings: 
M0= 
1 1 0 0

M1= 
1 0 1 0

M2= 
0 0 0 1

Pairwise mutual information: 
I(Z0; Z1) = 0.207519
I(Z0; Z2) = 0
I(Z1; Z2) = 0

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] ]
agglomerates to [ [ 0 1 ] [ 2 ] ] at critical value 0.207519
agglomerates to [ [ 0 1 2 ] ] at critical value -8.88178e-16

Approximate solution: 
critical values : [ 0.207519 0 ]
partition at threshold 0.207519:[ [ 0 ] [ 1 ] [ 2 ] ]
partition at threshold 0:[ [ 0 1 ] [ 2 ] ]


We can extend the *additive while gaussian noise (AWGN)* model to an arbitrary size:
\begin{align}
    \RZ_i &= \Rx_{k_i} + \sigma_i \Rx_{k + i} \kern 1em \text {for $i\in V$,}\label{eq:AWGN}
\end{align}
where 
- $V:=\Set{0,\dots,n-1}$ for some positive integer $n$ of nodes, 
- $k$ is a positive integer number of signals $\Rx_0,\dots,\Rx_{k-1}$, and
- for node $i\in V$
    - $k_i \in \Set{0,\dots,k-1}$ is the signal index of node $i$, and
    - $\sigma_i^2$ is the variance of the noise $\Rx_{k+i}$.

In [18]:
{
    size_t k = 5;
    double sigma = 1;
    size_t n = 15;

    // Define the linear mappings for the AWGN model
    Eigen::MatrixXd A = Eigen::MatrixXd::Zero(n, n + k);
    for (size_t i = 0; i < A.rows(); i++) {
        A(i, i % k) = 1;
        A(i, k + i) = sigma;
    }
    std::cout << "Z_V = M x where " << std::endl << "A= \n" << A << std::endl;
    std::vector<Eigen::MatrixXd> M(A.rows());
    for (size_t i = 0; i < A.rows(); i++) {
        M[i] = A.row(i);
    }
    
    IC::GaussianEntropy h(M);
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }
}

Z_V = M x where 
A= 
1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] [ 4 ] [ 5 ] [ 6 ] [ 7 ] [ 8 ] [ 9 ] [ 10 ] [ 11 ] [ 12 ] [ 13 ] [ 14 ] ]
agglomerates to [ [ 0 10 5 ] [ 1 11 6 ] [ 2 12 7 ] [ 3 13 8 ] [ 4 14 9 ] ] at critical value 0.25
agglomerates to [ [ 0 10 5 1 2 3 4 11 6 12 7 13 8 14 9 ] ] at critical value 7.10543e-15

Approximate solution: 
critical values : [ 0.207519 0 ]
partition at thres

Note that the exact and approximation solutions differ in the non-trivial critical value.

## Finite linear source

A finite linear source source model is defined as,
$$\RZ_i := \MM_i\RMx \qquad \text{for $i\in V$}$$
where $\RMx$ is a uniformly random row vector over a finite field $\mathbb{F}_{p^k}$ and $\MM_i$ can be any deterministic matrix.

The entropy function is given by
$$
\begin{align}
\begin{split}
h(B)&:=(k\log_2 p) \operatorname{rank} \MM_B\qquad \text{where}\\
\MM_B &:=\begin{bmatrix}\MM_{i_1} \\ \MM_{i_2}\\ \vdots\end{bmatrix}\kern1em \text{for any}\kern1em B=\Set{i_1,i_2,\dots}\subseteq V.
\end{split}
\end{align}
$$
$\MM$ is obtained by stacking the matrices $\MM_i$ for $i\in B$ vertically.  

To load the library for finite linear source:

*You may ignore the warnings about 'register' storage class specifier being deprecated and incompatible with C++17.*

In [20]:
#pragma cling add_include_path("$HOME/usr/include")
#pragma cling add_library_path("$HOME/usr/lib")
#pragma cling load("pari")
#include <IC/FLS>

The following is a simple example with $V:=\Set{0,1,2,3}$,
$$
\begin{aligned}
\RZ_0 &:= \Rx_0\\
\RZ_1 &:= \Rx_1\\
\RZ_2 &:= \Rx_0+\Rx_1\\
\RZ_3 &:=0
\end{aligned}\qquad
\text{where }
\RMx = \begin{bmatrix}\Rx_0\\ \Rx_1\end{bmatrix}
$$
is uniformly randomly over $\mathbb{F}_2^2.$

In [21]:
{
    int pField = 2;
    int kField = 1;

    size_t m = 3;
    size_t n0 = 1, n1 = 1, n2 = 1, n3 = 0;
    Eigen::MatrixXd M0(n0,m);
    Eigen::MatrixXd M1(n1,m);
    Eigen::MatrixXd M2(n2,m);
    Eigen::MatrixXd M3(n3,m);
    M0 << 1,0,0;
    M1 << 0,1,0;  
    M2 << 1,1,0;

    pari_init(1000000,2);

    // compute pairwise mutual information
    std::vector<Eigen::MatrixXd> LinearMappings = {M0,M1,M2,M3};
    IC::FiniteLinearEntropy h(LinearMappings,pField,kField);
    for (size_t i=0; i<h.size(); i++)
        for (size_t j=i+1; j<h.size(); j++)
        std::cout << "I(Z" << i << "; Z" << j << ") = " 
                  << h.mutual_info(i,j) << std::endl;    
    
    // generate exact info-clustering solution
    std::cout << std::endl << "Exact solution: " << std::endl;
    IC::AIC psp(h);
    std::cout << psp.getCoarsestPartition() 
              << std::endl; 
    while (psp.agglomerate()) 
            std::cout << "agglomerates to " << psp.getCoarsestPartition() 
                  << " at critical value " << psp.getCriticalValues().back() 
                  << std::endl;
    
    // generate approximate info-clustering solution
    std::cout << std::endl << "Approximate solution: " << std::endl;
    IC::CL cl(h);
    std::vector<double> cl_gamma = cl.getCriticalValues();
    std::cout << "critical values : " << cl_gamma << std::endl;
    for (double gamma : cl.getCriticalValues()) {
        std::cout << "partition at threshold " << gamma 
                  << ":" << cl.getPartition(gamma) << std::endl;
    }   
    
    pari_close();   
}

I(Z0; Z1) = 0
I(Z0; Z2) = 0
I(Z0; Z3) = 0
I(Z1; Z2) = 0
I(Z1; Z3) = 0
I(Z2; Z3) = 0

Exact solution: 
[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] ]
agglomerates to [ [ 0 2 1 ] [ 3 ] ] at critical value 0.5
agglomerates to [ [ 0 2 1 3 ] ] at critical value -0

Approximate solution: 
critical values : [ 0 ]
partition at threshold 0:[ [ 0 ] [ 1 ] [ 2 ] [ 3 ] ]


Note that $\RZ_i$ for $i\in V$ are pairwise independent, so the approximate info-clustering solution fails to identify the non-trivial cluster $\Set{0,1,2}$ with MMI $0.5$.